## Importing the modules

In [5]:
import pandas as pd
import requests
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import glob
import os
from pyproj import Transformer
import time
from scipy.spatial import cKDTree

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reading the Property sale data 2023

In [6]:
csv_file = "/content/drive/MyDrive/data science project/pp-2023.csv"
land_registry_df = pd.read_csv(csv_file, header=None)
print(land_registry_df.shape)

(812766, 16)


 ##### Link to the header file https://www.gov.uk/guidance/about-the-price-paid-data




In [7]:
land_registry_df.columns = ['Transaction unique identifier','Price','Date of Transfer',
                            'Postcode','PropertyType','Old/New','Duration','PAON','SAON',
                            'Street','Locality','Town/city','District','County','PPD Category Type','Record Status-monthly file only']

## Reading the Stops data



In [8]:
csv_file = "/content/drive/MyDrive/data science project/Stops.csv"
Stops = pd.read_csv(csv_file)
print(Stops.shape)

<ipython-input-8-ee71dce86309>:2: DtypeWarning: Columns (1,2,6,8,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  Stops = pd.read_csv(csv_file)


(435436, 43)


In [9]:
Stops[:5].columns

Index(['ATCOCode', 'NaptanCode', 'PlateCode', 'CleardownCode', 'CommonName',
       'CommonNameLang', 'ShortCommonName', 'ShortCommonNameLang', 'Landmark',
       'LandmarkLang', 'Street', 'StreetLang', 'Crossing', 'CrossingLang',
       'Indicator', 'IndicatorLang', 'Bearing', 'NptgLocalityCode',
       'LocalityName', 'ParentLocalityName', 'GrandParentLocalityName', 'Town',
       'TownLang', 'Suburb', 'SuburbLang', 'LocalityCentre', 'GridType',
       'Easting', 'Northing', 'Longitude', 'Latitude', 'StopType',
       'BusStopType', 'TimingStatus', 'DefaultWaitTime', 'Notes', 'NotesLang',
       'AdministrativeAreaCode', 'CreationDateTime', 'ModificationDateTime',
       'RevisionNumber', 'Modification', 'Status'],
      dtype='object')

## Reading the School data

In [10]:
csv_file2 = "/content/drive/MyDrive/data science project/2023-2024_england_school_information.csv"
schools = pd.read_csv(csv_file2)
schools.shape

(25489, 26)

## Data preprocessing



### Data type conversion

In [11]:
for col in ['Easting','Northing','Longitude','Latitude']:
    Stops[col] = pd.to_numeric(Stops[col], errors='coerce')

print(Stops[['CommonName','Longitude','Latitude']])

                               CommonName  Longitude   Latitude
0                        Temple Meads Stn  -2.582620  51.448980
1                        Temple Meads Stn  -2.581770  51.449440
2                        Temple Meads Stn  -2.585790  51.449020
3                        Temple Meads Stn  -2.585600  51.450140
4                        Temple Meads Stn  -2.584890  51.449670
...                                   ...        ...        ...
435431    Talbot Road (Blackpool Tramway)  -3.054507  53.819144
435432    Talbot Road (Blackpool Tramway)  -3.054537  53.819135
435433  North Station (Blackpool Tramway)  -3.050058  53.821035
435434  North Station (Blackpool Tramway)  -3.050088  53.821034
435435  North Station (Blackpool Tramway)  -3.050027  53.821035

[435436 rows x 3 columns]


In [12]:
land_registry_df["Price"] = pd.to_numeric(land_registry_df["Price"], errors="coerce")

print(land_registry_df[['Postcode','Price','Town/city','District','County']])

        Postcode   Price      Town/city       District          County
0       CF14 7BX  330000        CARDIFF        CARDIFF         CARDIFF
1       LL28 4SH  269950     COLWYN BAY          CONWY           CONWY
2       LL31 9BN  280000          CONWY          CONWY           CONWY
3       SA62 6BA  699999  HAVERFORDWEST  PEMBROKESHIRE   PEMBROKESHIRE
4       SY16 1QY  160000        NEWTOWN          POWYS           POWYS
...          ...     ...            ...            ...             ...
812761  LS12 4FJ  235000          LEEDS          LEEDS  WEST YORKSHIRE
812762   LS4 2EJ  229995          LEEDS          LEEDS  WEST YORKSHIRE
812763   WF7 5QE  253995     PONTEFRACT      WAKEFIELD  WEST YORKSHIRE
812764  LS12 4FL  270000          LEEDS          LEEDS  WEST YORKSHIRE
812765  LS25 4EP  279995          LEEDS          LEEDS  WEST YORKSHIRE

[812766 rows x 5 columns]


### Postcodes to latitude & Longitude conversion

#### link https://osdatahub.os.uk/downloads/open/CodePointOpen


In [13]:
folder_path = "/content/drive/MyDrive/data science project/CSV/"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []
for file in csv_files:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)


d=[]
for i in df_list[:-1]:
  d.append(i.values)
master_columns = ['Postcode','Positional_quality_indicator','Eastings','Northings','Countrycode','NHS_reg_HA_code','NHS_code','Admin_county',
                  'Admin_dist','Admin_ward']
combined_data = np.concatenate(d, axis=0)
df_combined = pd.DataFrame(combined_data,columns = master_columns )

<ipython-input-13-4ed9e8c58db8>:6: DtypeWarning: Columns (0,4,5,6,7,8,9,10,13,14,15,16,17,20,21,22,25,26,27,30,31,32,33,36,37,38,41,42,43,46,47,48,49,52,53,54,55,56,59,61,62,63,64,67,68,69,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file)


In [14]:
df_list[1]

,BR1 1AA,10,540291,168873,E92000001,E19000003,E18000007,Unnamed: 7,E09000006,E05013991
0,BR1 1AB,10,540262,169405,E92000001,E19000003,E18000007,NaN,E09000006,E05013991
1,BR1 1AE,10,540197,169204,E92000001,E19000003,E18000007,NaN,E09000006,E05013991
2,BR1 1AF,10,540259,168855,E92000001,E19000003,E18000007,NaN,E09000006,E05013991
3,BR1 1AG,10,540259,168855,E92000001,E19000003,E18000007,NaN,E09000006,E05013991
4,BR1 1AH,10,540430,168754,E92000001,E19000003,E18000007,NaN,E09000006,E05013991
...,...,...,...,...,...,...,...,...,...,...
6799,BR8 9DT,10,551403,168554,E92000001,E19000002,E18000008,E10000016,E07000111,E05005031
6800,BR8 9DU,10,551403,168554,E92000001,E19000002,E18000008,E10000016,E07000111,E05005031
6801,BR8 9EJ,10,551403,168554,E92000001,E19000002,E18000008,E10000016,E07000111,E05005031
6802,BR8 9EL,10,551403,168554,E92000001,E19000002,E18000008,E10000016,E07000111,E05005031


In [15]:
df_combined

,Postcode,Positional_quality_indicator,Eastings,Northings,Countrycode,NHS_reg_HA_code,NHS_code,Admin_county,Admin_dist,Admin_ward
0,B1 1BA,10,406875,286393,E92000001,E19000001,E18000005,NaN,E08000025,E05011151
1,B1 1BB,10,406654,286948,E92000001,E19000001,E18000005,NaN,E08000025,E05011151
2,B1 1BD,10,406689,286781,E92000001,E19000001,E18000005,NaN,E08000025,E05011151
3,B1 1BE,10,406721,286611,E92000001,E19000001,E18000005,NaN,E08000025,E05011151
4,B1 1BL,10,406779,286550,E92000001,E19000001,E18000005,NaN,E08000025,E05011151
...,...,...,...,...,...,...,...,...,...,...
1742600,TS9 7LL,10,458129,495552,E92000001,E19000001,E18000003,NaN,E06000065,E05014276
1742601,TS9 7LN,10,458201,496162,E92000001,E19000001,E18000003,NaN,E06000065,E05014276
1742602,TS9 7LQ,10,457418,493546,E92000001,E19000001,E18000003,NaN,E06000065,E05014276
1742603,TS9 7LW,10,458229,497257,E92000001,E19000001,E18000003,NaN,E06000065,E05014276


In [16]:
transformer = Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)

coor= []
for i,j in df_combined[['Eastings','Northings']].values:
  lon, lat = transformer.transform(i, j)
  coor.append([lon,lat])

new_columns = pd.DataFrame(coor, columns=['Longitude_prop', 'Latitude_prop'])

rep = pd.concat([new_columns] * (len(df_combined) // len(new_columns) + 1), ignore_index=True).head(len(df_combined))
all_postcodes = pd.concat([df_combined, rep], axis=1)

In [17]:
df_merged = land_registry_df.merge(all_postcodes[['Postcode','Longitude_prop','Latitude_prop']], on='Postcode', how='inner')

print("\nMerged DataFrame (Inner Join):")
print(df_merged)


Merged DataFrame (Inner Join):
                 Transaction unique identifier   Price  Date of Transfer  \
0       {06C9F487-D94B-9388-E063-4804A8C0BD98}  330000  2023-03-20 00:00   
1       {06C9F487-D94C-9388-E063-4804A8C0BD98}  269950  2023-07-25 00:00   
2       {06C9F487-D94D-9388-E063-4804A8C0BD98}  280000  2023-08-10 00:00   
3       {06C9F487-D94E-9388-E063-4804A8C0BD98}  699999  2023-08-24 00:00   
4       {06C9F487-D94F-9388-E063-4804A8C0BD98}  160000  2023-08-21 00:00   
...                                        ...     ...               ...   
809924  {152AB734-65BF-E651-E063-4704A8C061D9}  235000  2023-03-07 00:00   
809925  {152AB734-65C0-E651-E063-4704A8C061D9}  229995  2023-04-21 00:00   
809926  {152AB734-65C2-E651-E063-4704A8C061D9}  253995  2023-04-24 00:00   
809927  {152AB734-65C3-E651-E063-4704A8C061D9}  270000  2023-03-24 00:00   
809928  {152AB734-65C4-E651-E063-4704A8C061D9}  279995  2023-05-19 00:00   

        Postcode PropertyType Old/New Duration         

In [18]:
schools.rename(columns={'POSTCODE': 'Postcode'}, inplace=True)

In [19]:
school_merged = schools.merge(all_postcodes[['Postcode','Longitude_prop','Latitude_prop']], on='Postcode', how='inner')

print("\nMerged DataFrame (Inner Join):")
print(school_merged)


Merged DataFrame (Inner Join):
          URN           LANAME   LA  ESTAB  LAESTAB  \
0      100000   City of London  201   3614  2013614   
1      100001   City of London  201   6005  2016005   
2      100002   City of London  201   6006  2016006   
3      100003   City of London  201   6007  2016007   
4      100008           Camden  202   2019  2022019   
...       ...              ...  ...    ...      ...   
25393  151399  North Yorkshire  815   2225  8152225   
25394  151432          Norfolk  926   2276  9262276   
25395  151443        Blackpool  890   4007  8904007   
25396  151533        Wiltshire  865   4009  8654009   
25397  151534            Essex  881   2195  8812195   

                                         SCHNAME                     STREET  \
0                             The Aldgate School         St James's Passage   
1                City of London School for Girls          St Giles' Terrace   
2                     St Paul's Cathedral School               2 New C

### Checking the null values

In [20]:
print('_________ Land registry merged data __________\n')
print(df_merged.isnull().sum())
print('\n ___________ Stops ______________\n')
print(Stops.isnull().sum())
print('\n _______________   School merged data ____________\n')
print(school_merged.isnull().sum())

_________ Land registry merged data __________

Transaction unique identifier           0
Price                                   0
Date of Transfer                        0
Postcode                                0
PropertyType                            0
Old/New                                 0
Duration                                0
PAON                                    0
SAON                               703022
Street                              12927
Locality                           506337
Town/city                               0
District                                0
County                                  0
PPD Category Type                       0
Record Status-monthly file only         0
Longitude_prop                          0
Latitude_prop                           0
dtype: int64

 ___________ Stops ______________

ATCOCode                        0
NaptanCode                  25721
PlateCode                  372481
CleardownCode              435436
CommonName 

In [21]:
Stops.dropna(subset = ['Latitude', 'Longitude'], how='all',inplace=True)

### Top 30 Districts

In [22]:
top_30 = pd.DataFrame(df_merged['District'].value_counts()[:30]).reset_index()
locations = top_30['District'].values

In [23]:
filtered_df = df_merged[df_merged['District'].isin(locations)]

### Creating New columns

### DISTANCE calculation betweeen Stops data and land registry data

In [24]:
def latlon_to_cartesian(lat, lon, R=6371.0):
    lat_radians = np.radians(lat)
    lon_radians = np.radians(lon)
    x = R * np.cos(lat_radians) * np.cos(lon_radians)
    y = R * np.cos(lat_radians) * np.sin(lon_radians)
    z = R * np.sin(lat_radians)
    return x, y, z

In [25]:
# Build station coords & KDTree
sx, sy, sz = latlon_to_cartesian(Stops['Latitude'], Stops['Longitude'])
station_coords = np.column_stack((sx, sy, sz))
tree = cKDTree(station_coords)

In [26]:

# Convert properties to 3D
# FILTERED_DF IS THE LAND REGISTRY FINAL DATA
px, py, pz = latlon_to_cartesian(filtered_df['Latitude_prop'], filtered_df['Longitude_prop'])
prop_coords = np.column_stack((px, py, pz))

In [27]:
# Query tree
distances, indices = tree.query(prop_coords, k=1)

# Attach results
filtered_df['nearest_station_index'] = indices
filtered_df['dist_chord_km'] = distances
filtered_df['closest_station'] = filtered_df['nearest_station_index'].apply(
    lambda i: Stops.iloc[i]['CommonName'] if i >= 0 else None
)


<ipython-input-27-385ca91d814f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['nearest_station_index'] = indices
<ipython-input-27-385ca91d814f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dist_chord_km'] = distances
<ipython-input-27-385ca91d814f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

###  DISTANCE calculation betweeen Schools data and land registry data

In [28]:
# Build station coords & KDTree
sx1, sy1, sz1 = latlon_to_cartesian(school_merged['Latitude_prop'], school_merged['Longitude_prop'])
school_coords = np.column_stack((sx1, sy1, sz1))
tree1 = cKDTree(school_coords)

In [29]:
# Convert properties to 3D
px1, py1, pz1 = latlon_to_cartesian(filtered_df['Latitude_prop'], filtered_df['Longitude_prop'])
prop_coords1 = np.column_stack((px1, py1, pz1))

In [30]:
# Query tree
distances1, indices1 = tree1.query(prop_coords1, k=1)

# Attach results
filtered_df['nearest_school_index'] = indices1
filtered_df['dist_school_km'] = distances1
filtered_df['closest_school'] = filtered_df['nearest_school_index'].apply(
    lambda i: school_merged.iloc[i]['SCHNAME'] if i >= 0 else None
)

<ipython-input-30-7d80b88ed0f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['nearest_school_index'] = indices1
<ipython-input-30-7d80b88ed0f8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dist_school_km'] = distances1
<ipython-input-30-7d80b88ed0f8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [31]:
filtered_df.head()
filtered_df.to_csv("/content/drive/MyDrive/data science project/final registry data.csv")